In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
import ms_analysis as msa
mc = msa.MSout("../ms_out/pH5eH0ms.txt")

In [3]:
ms_orig_lst = [[ms.E, ms.count, ms.state] for  ms in list((mc.microstates.values()))]
ms_orig_lst = sorted(ms_orig_lst, key = lambda x:x[0])

In [4]:
# This will convert the charge microstate id to charge id.
id_vs_charge = {}
for conf in msa.conformers:
    id_vs_charge[conf.iconf] = conf.crg

def convert_ms_crg(l, d):
    crg_lst =[[y[0], y[1], [convert_ms_crg(x, d) if isinstance(x, list) else d.get(x, x) for x in y[2]]] for y in l]
    return crg_lst

crg_orig_lst = convert_ms_crg(ms_orig_lst, id_vs_charge )


In [5]:
# this is to write the fixed charge state
fixed_res_crg_dict = {}
# list of titrable residues and your interest
titra_res = ["ASP", "GLU", "ARG", "HIS", "LYS", "CYS", "TYR", "NTR", "CTR"]

for conf in msa.conformers:
    if conf.iconf in mc.fixed_iconfs and  conf.resid[:3] in titra_res:
        try: 
            if conf.resid not in fixed_res_crg_dict:
                fixed_res_crg_dict [conf.resid] = conf.crg
        except:
            print("Fixed residues are duplicating")


fixed_residues_crg = pd.DataFrame(fixed_res_crg_dict.items(), columns=['Residue', 'crg'])

print(sum(fixed_res_crg_dict.values()))

fixed_residues_crg.head() 

16.0


,Residue,crg
0,ARGA0005_,1.0
1,LYSA0013_,1.0
2,ARGA0014_,1.0
3,ARGA0021_,1.0
4,TYRA0023_,0.0


In [17]:
# This will  write out the pdb file  from conformer ms state. 
# input will take the list of ms in the form of ms_orig_lst i.e. third memeber is the list of ms.

import os
import shutil
def write_ms2_pdb(ms_list, ms_start =0, skip_step_ms = 0, output_folder = 'ms_pdb'):
    
    if os.path.exists(output_folder):
        shutil.rmtree(output_folder)
    os.makedirs(output_folder)
    
    # this is to read the ms list file
    for x in range(ms_start, len(ms_list), skip_step_ms+1):
        all_conf_list = []
        for conf in msa.conformers:
            if conf.iconf in ms_list[x][2] or conf.iconf in mc.fixed_iconfs:
        #print(conf.confid)
                all_conf_list.append(conf.confid)
        #print(all_conf_list)
        # read the step2_file
        pdb = open("../step2_out.pdb").readlines()
        file_name = f"{output_folder}/ms_pdb_{x}.pdb"
        with open(file_name, 'w') as output_pdb:
            for line in pdb:
                if len(line) <82: continue
                if line[26] == ' ': iCode = '_'
                else: iCode = line[26]
                confID = line[17:20]+line[80:82]+line[21:26]+iCode+line[27:30]

                if confID in all_conf_list or confID[3:5] == 'BK':
                        output_pdb.write(line)

    print(f"Skiping {skip_step_ms} microstate")
    return   

In [9]:
# writing out pdb using lowest, average and highest ms energy
# from the origin list. Select the  two lowest energy conformer ms, two highest energy conformer ms state and  
# and 5 random  around average energy state ms state(+/- 0.0001) .
#pdb_crg_orig_lst = crg_orig_lst.copy()
import random
random.seed(100)
ms_orig_lst_copy = ms_orig_lst.copy()
pdb_low_high_conf_ms = [ms_orig_lst_copy[0], ms_orig_lst_copy[1], ms_orig_lst_copy[-2], ms_orig_lst_copy[-1]]
pdb_av_conf_ms_random = random.sample([[x[0], x[1], x[2]] for x in ms_orig_lst_copy \
                                      if x[0] >= (mc.average_E  - 0.0001) and  x[0] <= (mc.average_E  + 0.0001)], 5)

pdb_low_high_av_conf_ms = pdb_low_high_conf_ms + pdb_av_conf_ms_random
#print(pdb_low_high_av_conf_ms)
print(len(pdb_low_high_av_conf_ms))

9


In [18]:
# write out the pdb in the folder low_high_avg_pdb
write_ms2_pdb(pdb_low_high_av_conf_ms, ms_start =0, skip_step_ms = 0, output_folder = "low_high_avg_pdb")


Skiping 0 microstate


In [8]:
# writing the ms pdb file based on count and unique ms state. so
# This will find the unique charge of all charge microstate. Three unique charge state with highest count will select for MD.
# This will add to the dictionary value and make a list of enthalpy with maximum and minimum value
# begin_energy = crg_list_ms[0][0], end_energy = crg_list_ms[-1][0]
def find_unique_crg_ms(crg_list_ms, begin_energy = mc.lowest_E, end_energy = mc.highest_E):
    crg_list_ms = [[x[0], x[1], x[2]] for x in crg_list_ms if x[0] >= begin_energy and x[0] <= end_energy]
    # this is for unique charge ms. Each key is the charge ms and values are list of count and maximum and minimum
    # energy
    which_position_in_ms_state = []
    crg_all_count = {}
    for x, array in enumerate(crg_list_ms):
        if tuple(array[2]) not in crg_all_count.keys():
            crg_all_count[(tuple(array[2]))] = [array[1], [array[0]]]
            which_position_in_ms_state.append([x,array[2]])

        else:
            crg_all_count[(tuple(array[2]))][0] += array[1]
            if len(crg_all_count[(tuple(array[2]))][1]) == 1:
                if array[0] < crg_all_count[(tuple(array[2]))][1][0]:
                    crg_all_count[(tuple(array[2]))][1].insert(0, array[0])


                else:
                    crg_all_count[(tuple(array[2]))][1].append(array[0])

            else:
                if array[0] < crg_all_count[(tuple(array[2]))][1][0]:
                    del crg_all_count[(tuple(array[2]))][1][0]
                    crg_all_count[(tuple(array[2]))][1].insert(0, array[0])
                elif array[0] > crg_all_count[(tuple(array[2]))][1][-1]:
                    crg_all_count[(tuple(array[2]))][1].pop()
                    crg_all_count[(tuple(array[2]))][1].append(array[0])
                else:
                    continue
    # make a list of count and charge microstate
    all_crg_ms_unique  = []
    all_count = []
    energy_diff_all = []
    for u,v in crg_all_count.items():
        all_crg_ms_unique.append(list(u))
        all_count.append(v[0])
        if len(v[1]) == 2:
            energy_diff_all.append(round(v[1][1]-v[1][0], 6))
        elif len(v[1]) == 1:
            energy_diff_all.append(0)
        else:
            print("error in dictionary")
    return all_crg_ms_unique, all_count, energy_diff_all, which_position_in_ms_state


In [12]:
#all_crg_ms_unique, all_count,  energy_diff_all, which_position_in_ms_state 
all_crg_ms = find_unique_crg_ms(crg_orig_lst,begin_energy = crg_orig_lst[0][0], end_energy = crg_orig_lst[-1][0])

In [14]:
# let us make the list of  charge unique ms, count and sum_crg from the unique charge ms.
pdb_unique_crg_ms_count = [list(i) for i in zip(all_crg_ms[1], all_crg_ms[0])]
pdb_unique_crg_ms_count_sum = [[x[0],sum(fixed_res_crg_dict.values())+ sum(x[1]), x[1]] for x in pdb_unique_crg_ms_count]
#unique_crg_ms_count_sum = sorted(unique_crg_ms_count_sum, key = lambda x:-x[0])
print(len(pdb_unique_crg_ms_count_sum))
#print(pdb_unique_crg_ms_count_sum)


147


In [15]:
# we know the first postion of unique charge and ms state from  which_position_in_ms_state. Let us try to make list that
# has count, charge and microstate position.
# match the charge set from unique to get count. Now only match both unique length and take ms conformers microstate
def match_unique_ms(list_1, list_2):
    count_sum_position = []
    # list_1 is pdb_from unique_crg_ms_count_sum
    # list_2 is which_position_in_ms_state from function find_unique_crg_ms
    x, y = 0, 0
    while x < len(list_1):
        if list_1[x][2] == list_2[y][1]:
        #print(unique_crg_ms_count_sum[x][2])
        
            count_sum_position.append([list_1[x][0], list_1[x][1], list_2[y][0]])
            x += 1
            y = 0
        
        else:
            y += 1
        
    return count_sum_position

# this is to filter out conf ms state.
#pdb_conf_ms_count =[[ms_orig_lst[x[2]]] for x in  match_unique_ms(pdb_unique_crg_ms_count_sum, all_crg_ms[3])] 
pdb_ms_count_crg_conf = [[x[0], x[1], ms_orig_lst[x[2]][2]] for x in match_unique_ms(pdb_unique_crg_ms_count_sum, all_crg_ms[3])]
# sort based on count. Highest count first.
pdb_ms_count_crg_conf  = sorted(pdb_ms_count_crg_conf, key=  lambda x: -x[0])
print(pdb_ms_count_crg_conf[:5])

[[647121, 10.0, [1, 7, 20, 36, 42, 43, 46, 55, 58, 69, 79, 81, 83, 86, 92, 94, 104, 112, 116, 123, 125, 129, 132, 137, 146, 154, 160, 174, 185, 188, 198, 203, 211, 223, 228, 229, 233, 236, 245, 254, 260, 263, 283]], [510091, 9.0, [1, 7, 20, 36, 42, 43, 46, 55, 58, 72, 79, 81, 83, 86, 92, 94, 104, 112, 116, 123, 125, 129, 132, 137, 146, 154, 160, 174, 185, 188, 198, 203, 211, 223, 228, 229, 233, 236, 245, 254, 260, 263, 283]], [67367, 11.0, [1, 7, 20, 36, 42, 43, 46, 55, 58, 69, 79, 81, 83, 86, 92, 94, 105, 112, 116, 123, 125, 129, 132, 137, 146, 154, 160, 174, 184, 188, 198, 203, 211, 223, 224, 229, 233, 236, 245, 254, 260, 263, 283]], [57598, 10.0, [1, 6, 20, 36, 42, 43, 46, 55, 58, 72, 79, 81, 83, 86, 92, 94, 104, 112, 116, 123, 125, 129, 132, 137, 146, 154, 162, 174, 185, 188, 198, 203, 211, 223, 227, 229, 233, 236, 245, 254, 260, 263, 283]], [33380, 9.0, [0, 7, 20, 36, 42, 43, 46, 55, 58, 69, 78, 81, 83, 86, 92, 94, 105, 112, 116, 123, 125, 129, 132, 137, 146, 154, 160, 174, 184, 1

In [19]:
# define here how many ms states you want to skip. This will write the pdb in the form of step2_out for each ms state. If your
# system is large then please do not write every pdb otherwise it will raise the memory issue. Give the directory name you 
# want to create and save the files.
write_ms2_pdb(pdb_ms_count_crg_conf, ms_start =0, skip_step_ms = 100, output_folder = "count_pdb")

Skiping 100 microstate
